In [26]:
import numpy as np
import pandas as pd
from xgboost import XGBClassifier
from sklearn.metrics import matthews_corrcoef, roc_auc_score

ImportError: cannot import name 'cmaps' from 'matplotlib._cm_listed' (F:\Users\Conda\DSDefault\lib\site-packages\matplotlib\_cm_listed.py)

In [2]:
date_chunks = pd.read_csv("F:\\Pycharm\\SCDS-borchFactory\\data\\train_date.csv", index_col=0, chunksize=100000, dtype=np.float32)
num_chunks = pd.read_csv("F:\\Pycharm\\SCDS-borchFactory\\data\\train_numeric.csv", index_col=0,
                         usecols=list(range(969)), chunksize=100000, dtype=np.float32)
X = pd.concat([pd.concat([dchunk, nchunk], axis=1).sample(frac=0.05)
               for dchunk, nchunk in zip(date_chunks, num_chunks)])
y = pd.read_csv("F:\\Pycharm\\SCDS-borchFactory\\data\\train_numeric.csv", index_col=0, usecols=[0,969], dtype=np.float32).loc[X.index].values.ravel()
X = X.values

F:\Users\Conda\DSDefault\lib\site-packages\numpy\lib\arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [3]:
X

array([[1530.91, 1530.91, 1530.91, ...,     nan,     nan,     nan],
       [    nan,     nan,     nan, ...,     nan,     nan,     nan],
       [ 672.65,  672.65,  672.65, ...,     nan,     nan,     nan],
       ...,
       [1451.61, 1451.61, 1451.61, ...,    0.  ,    0.  ,    0.  ],
       [ 912.84,  912.84,  912.84, ...,     nan,     nan,     nan],
       [    nan,     nan,     nan, ...,    0.  ,    0.  ,    0.  ]],
      dtype=float32)

In [4]:
print('done')

done


In [5]:
clf = XGBClassifier(base_score=0.005)
clf.fit(X, y)

XGBClassifier(base_score=0.005, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [65]:
#plt.hist(clf.feature_importances_[clf.feature_importances_>0])
important_indices = np.where(clf.feature_importances_>0.005)[0]
print(important_indices)

[ 883  948 1018 1019 1056 1160 1173 1183 1193 1222 1227 1245 1282 1298
 1480 1512 1516 1549 1550 1893 1897 1921 1925 1968 1970 2007 2010 2020
 2022 2040 2067 2100]


In [ ]:
# load entire dataset for these features. 
# note where the feature indices are split so we can load the correct ones straight from read_csv
n_date_features = 1156
X = np.concatenate([
    pd.read_csv("../input/train_date.csv", index_col=0, dtype=np.float32,
                usecols=np.concatenate([[0], important_indices[important_indices < n_date_features] + 1])).values,
    pd.read_csv("../input/train_numeric.csv", index_col=0, dtype=np.float32,
                usecols=np.concatenate([[0], important_indices[important_indices >= n_date_features] + 1 - 1156])).values
], axis=1)
y = pd.read_csv("../input/train_numeric.csv", index_col=0, dtype=np.float32, usecols=[0,969]).values.ravel()

In [58]:
print(X.shape)

(59187, 2124)


In [42]:
from sklearn.model_selection import StratifiedKFold
clf = XGBClassifier(max_depth=5, base_score=0.005)
cv = StratifiedKFold(n_splits=3, shuffle=False,random_state=None)
preds = np.ones(y.shape[0])

In [61]:
for (train, test) in cv.split(X,y):
    preds[test] = clf.fit(X[train], y[train]).predict_proba(X[test])[:,1]
    print("fold {}, ROC AUC: {:.3f}".format(roc_auc_score(y[test], preds[test])))
print(roc_auc_score(y, preds))

fold [11683 11761 11840 ... 59184 59185 59186], ROC AUC: 0.671
fold [11683 11761 11840 ... 59184 59185 59186], ROC AUC: 0.662
fold [11683 11761 11840 ... 59184 59185 59186], ROC AUC: 0.667
fold [11683 11761 11840 ... 59184 59185 59186], ROC AUC: 0.733
fold [11683 11761 11840 ... 59184 59185 59186], ROC AUC: 0.654
0.6768796519631072


In [62]:
# pick the best threshold out-of-fold
thresholds = np.linspace(0.01, 0.99, 50)
mcc = np.array([matthews_corrcoef(y, preds>thr) for thr in thresholds])
#plt.plot(thresholds, mcc)
best_threshold = thresholds[mcc.argmax()]
print(mcc.max())

F:\Users\Conda\DSDefault\lib\site-packages\sklearn\metrics\_classification.py:900: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)


0.1488404342392704


In [75]:
# load test data
X = np.concatenate([
    pd.read_csv("F:\\Pycharm\\SCDS-borchFactory\\data\\test_date.csv", index_col=0, dtype=np.float32,
                usecols=np.concatenate([[0], important_indices[important_indices<1156]+1])).values,
    pd.read_csv("F:\\Pycharm\\SCDS-borchFactory\\data\\test_numeric.csv", index_col=0, dtype=np.float32,
                usecols=np.concatenate([[0], important_indices[important_indices>=1156] +1 - 1156])).values
], axis=1)

In [76]:
len(X)

1183748

In [77]:
# generate predictions at the chosen threshold
preds = (clf.predict_proba(X)[:,1] > best_threshold).astype(np.int8)

ValueError: feature_names mismatch: ['f0', 'f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9', 'f10', 'f11', 'f12', 'f13', 'f14', 'f15', 'f16', 'f17', 'f18', 'f19', 'f20', 'f21', 'f22', 'f23', 'f24', 'f25', 'f26', 'f27', 'f28', 'f29', 'f30', 'f31', 'f32', 'f33', 'f34', 'f35', 'f36', 'f37', 'f38', 'f39', 'f40', 'f41', 'f42', 'f43', 'f44', 'f45', 'f46', 'f47', 'f48', 'f49', 'f50', 'f51', 'f52', 'f53', 'f54', 'f55', 'f56', 'f57', 'f58', 'f59', 'f60', 'f61', 'f62', 'f63', 'f64', 'f65', 'f66', 'f67', 'f68', 'f69', 'f70', 'f71', 'f72', 'f73', 'f74', 'f75', 'f76', 'f77', 'f78', 'f79', 'f80', 'f81', 'f82', 'f83', 'f84', 'f85', 'f86', 'f87', 'f88', 'f89', 'f90', 'f91', 'f92', 'f93', 'f94', 'f95', 'f96', 'f97', 'f98', 'f99', 'f100', 'f101', 'f102', 'f103', 'f104', 'f105', 'f106', 'f107', 'f108', 'f109', 'f110', 'f111', 'f112', 'f113', 'f114', 'f115', 'f116', 'f117', 'f118', 'f119', 'f120', 'f121', 'f122', 'f123', 'f124', 'f125', 'f126', 'f127', 'f128', 'f129', 'f130', 'f131', 'f132', 'f133', '

In [ ]:
# and submit
sub = pd.read_csv("F:\\Pycharm\\SCDS-borchFactory\\data\\sample_submission.csv", index_col=0)
sub["Response"] = preds
sub.to_csv("submission.csv.gz", compression="gzip")

In [ ]:
print('done')
print(len(preds), len(sub))

In [ ]:
date_chunks = pd.read_csv("F:\\Pycharm\\SCDS-borchFactory\\data\\test_date.csv", index_col=0, dtype=np.float32,
                usecols=np.concatenate([[0], important_indices[important_indices<1156]+1]))
num_chunks = pd.read_csv("F:\\Pycharm\\SCDS-borchFactory\\data\\test_numeric.csv")
X = pd.concat([pd.concat([dchunk, nchunk], axis=1).sample(frac=0.05)
               for dchunk, nchunk in zip(date_chunks, num_chunks)])
y = pd.read_csv("F:\\Pycharm\\SCDS-borchFactory\\data\\train_numeric.csv", index_col=0, usecols=[0,969], dtype=np.float32).loc[X.index].values.ravel()
X = X.values



In [ ]:
print('done')

